## Managing the transaction lifecycle on LUSID

In this notebook we show how you can use `properties` to manage the transaction lifecycle. For the purposes of this demonstration, we're using the [transaction](https://support.finbourne.com/what-is-a-transaction) entity but the same principles could be applied to LUSID's orders and allocations, or indeed any data entity which has `properties`. In the example, we will add a <b>confirmation_status</b> and <b>settlement_status</b> to our transaction. In practise you can add as many statuses as you need. A quick note on terminology:

* <b>Confirmation</b>: The process by which the two counterparties to a trade input their instructions to a central system which compares them and, if the instructions agree, confirms them and passes them on for settlement.
* <b>Settlement</b>: The process of transferring securities into the account of a buyer and cash into the seller's account following a trade of stocks, bonds, futures or other financial assets.



### Setup LUSID

In [1]:
# Import LUSID
import lusid.models as models
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import pandas as pd
import numpy as np
import json
import requests
import os
import lusid
import lusidtools.cocoon.cocoon as cocoon
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
import uuid

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID API Version:  0.5.4277.0


In [2]:
# define a unqique scope
scope = create_scope_id()
portfolio_code = "EQUITY_UK"

# Load a mapping file for loading data
with open(r"config/transaction-lifecycle/transaction_status.json") as mappings_file:
    transaction_status_mapping = json.load(mappings_file)
    
# Load a mapping file for formatting DataFrame
with open(r"config/transaction-lifecycle/get_transactions_mapping.json") as mappings_file:
    get_transactions_df_mapping = json.load(mappings_file)

### 1) Introduction to transaction statuses


In the code below, we're going to create a confirmation and settlement status with some values which can be assigned to a `transaction`. As with all LUSID properties, you're free to assign your own codes and values. The below is a sample implementation. The values and codes are not fixed. 




<u><i><center> Table showing sample confirmation statuses </center></i></u>


| Confirmation status  | Status description |
| :------------- | :------------- |
|Unconfirmed |Trade is unconfirmed and no confirmation message has been sent to broker |
|MessageSentToBroker | Trade is unconfirmed and confirmation message has been sent to broker |
|AckRecFromBroker | Acknowledgement message has been recieved from broker |
|TradeConfirmed| Trade has confirmed | 
|FailedConfirmation | Trade confirmation has failed |


<br>
<br>


<u><i><center> Table showing sample settlement statuses </center></i></u>


| Settlement status  | Status description |
| :------------- | :------------- |
|Unsettled |Trade has not settled and no settlement message has been sent to custodian |
|MessageSentToCustodian | Trade is unsettled and settlement message has been sent to custodian |
|AckRecFromCustodian | Acknowledgement message has been recieved from custodian |
|Settled| Trade has settled | 
|FailedSettlement| Trade settlement has failed |

<br>
<br>

### 2) Create a new data type to hold acceptable values for the status

In this section, we create a new LUSID data type object to hold the list of acceptable values for each status:

* Acceptable values for confirmation status are: Unconfirmed, MessageSentToBroker, AckRecFromBroker, TradeConfirmed, and FailedConfirmation.
* Acceptable values for settlement status are: Unsettled, MessageSentToCustodian, AckRecFromCustodian, Settled, and FailedSettlement.

his data type will be used when creating the <b>confirmation_status</b> and <b>settlement_status</b> property definition.

In [3]:
# define lists for acceptable status values

confirmation_values = [
    "Unconfirmed",
    "MessageSentToBroker",
    "AckRecFromBroker",
    "TradeConfirmed",
    "FailedConfirmation",
]
settlement_values = [
    "Unsettled",
    "MessageSentToCustodian",
    "AckRecFromCustodian",
    "Settled",
    "FailedSettlement",
]

In [4]:
for status_code, values_list in [
    ("ConfirmationStatusCodes", confirmation_values),
    ("SettlementStatusCodes", settlement_values),
]:

    try:

        create_request = lusid.models.CreateDataTypeRequest(
            scope=scope,
            code=status_code,
            type_value_range="Closed",
            display_name=f"Available {status_code}",
            description=f"List of allowable values for {status_code}",
            value_type="String",
            acceptable_values=values_list,
        )

        response = api_factory.build(lusid.api.DataTypesApi).create_data_type(
            request=create_request
        )

        print(f"Data Type of {status_code} has been created.")
        print(f"The acceptable values for this data type are: {str(values_list)}\n")

    except:

        response = api_factory.build(lusid.api.DataTypesApi).get_data_type(
            scope=scope, code=status_code
        )

        print(response)

Data Type of ConfirmationStatusCodes has been created.
The acceptable values for this data type are: ['Unconfirmed', 'MessageSentToBroker', 'AckRecFromBroker', 'TradeConfirmed', 'FailedConfirmation']

Data Type of SettlementStatusCodes has been created.
The acceptable values for this data type are: ['Unsettled', 'MessageSentToCustodian', 'AckRecFromCustodian', 'Settled', 'FailedSettlement']



### 3) Create a new property definitions

Next we define a new <b>confirmation_status</b> and <b>settlement_status</b> property which will be used on the transaction to set the allowed statuses. As you can see, we have passed the <b>data_type_id</b> which we created above.

In [5]:
for status_code, data_type in [
    ("confirmation_status", "ConfirmationStatusCodes"),
    ("settlement_status", "SettlementStatusCodes"),
]:

    try:

        property_response = api_factory.build(
            lusid.api.PropertyDefinitionsApi
        ).create_property_definition(
            lusid.models.CreatePropertyDefinitionRequest(
                domain="Transaction",
                scope=scope,
                code=status_code,
                value_required=None,
                display_name=status_code,
                data_type_id=lusid.ResourceId(scope=scope, code=data_type),
                life_time=None,
            )
        )

    except:
        pass

### 4) Create new portfolio with transactions

In the code below, we create a new portfolio called <b>EQUITY_UK</b> with two `transactions` from the <i>equity_transactions.csv</i> file. There is one equity trade and one subsciption of cash. The `transactions` have the <b>transaction_status</b> of <i>Executed</i> and <i>Confirmed</i> which are allowed values per the data type above.

In [6]:
transactions_file = r"data/transaction-lifecycle/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code
transactions_df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,confirmation_status,settlement_status,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,02/01/2020,04/01/2020,120000,5,600000,GBP,MessageSentToBroker,MessageSentToCustodian,NaN
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,trd_0021,FundsIn,02/01/2020,04/01/2020,12000000,1,12000000,GBP,TradeConfirmed,Settled,GBP


In [7]:
# The seed_data() function takes a file of transaction data
# and loads portfolios, instruments, and transactions into LUSID
# We use this function as a quick way of generating a demo portfolio

seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
    mappings=transaction_status_mapping,
)

print(f"Portfolio {portfolio_code} has been created with transactions.")

Portfolio EQUITY_UK has been created with transactions.


### 5) Check transactions were loaded successfully

Check that the `transactions` loaded correctly by calling the [get transactions](https://www.lusid.com/docs/api/#operation/GetTransactions) method on the TransactionPortfolios API.

In [8]:
# Define the transactions portfolio API
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

# Call the get transactions method
get_transaction_response = transaction_portfolio_api.get_transactions(
    scope=scope, code=portfolio_code
)

# Scrape the response into a DataFrame
get_transactions_df = lusid_response_to_data_frame(
    get_transaction_response,
    rename_properties=True,
    column_name_mapping = get_transactions_df_mapping
)

# Print the DataFrame
get_transactions_df

,TransactionId,TransactionType,ClientId,LusidInstrumentId,TransactionDate,SettlementDate,Units,Price,PriceType,TotalConsideration,TotalConsiderationCurrency,ExchangeRate,TxnCurrency,SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),confirmation_status(385d-4ea5-2f28-7c-Properties),settlement_status(385d-4ea5-2f28-7c-Properties),InstrumentCurrency
0,trd_0001,Buy,EQ_1234,LUID_ATFGUBHS,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,120000.0,5.0,Price,600000.0,GBP,1.0,GBP,EQUITY_UK,385d-4ea5-2f28-7c,MessageSentToBroker,MessageSentToCustodian,NaN
1,trd_0021,FundsIn,NaN,CCY_GBP,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,12000000.0,1.0,Price,12000000.0,GBP,1.0,GBP,EQUITY_UK,385d-4ea5-2f28-7c,TradeConfirmed,Settled,GBP


### 6) Correct the confirmation status to <i>Matched</i> - this is an invalid value 

To show the data validation on <b>confirmation_status</b> field, we now want to add an unacceptable value. For the purposes of this demo, we will correct the <b>confirmation_status</b> to <i>Matched</i> which is not one of the 4 acceptable values.

In [9]:
transactions_df.at[0, "confirmation_status"] = "Matched"
transactions_df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,confirmation_status,settlement_status,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,Buy,02/01/2020,04/01/2020,120000,5,600000,GBP,Matched,MessageSentToCustodian,NaN
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GBP,GBP,cash,GBP,GBP Cash,trd_0021,FundsIn,02/01/2020,04/01/2020,12000000,1,12000000,GBP,TradeConfirmed,Settled,GBP


As you can see, the request fails with an <b>InvalidTransactions</b> error: 

In [10]:
result = cocoon.load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=transactions_df,
    mapping_required=transaction_status_mapping["transactions"]["required"],
    mapping_optional=transaction_status_mapping["transactions"]["optional"],
    file_type="transactions",
    identifier_mapping=transaction_status_mapping["transactions"]["identifier_mapping"],
    property_columns=transaction_status_mapping["transactions"]["properties"],
    properties_scope=scope,
)

error_response = result["transactions"]["errors"][0].body
print("Error type: ", json.loads(error_response)["name"], "\n")
print(
    "Transaction ID which created error: ",
    json.loads(error_response)["errorDetails"][0]["id"],
    "\n",
)
print("Error details: ", json.loads(error_response)["errorDetails"][0]["detail"], "\n")

Error type:  InvalidTransactions 

Transaction ID which created error:  trd_0001 

Error details:  Invalid value 'Matched' supplied to the field of data type 385d-4ea5-2f28-7c/ConfirmationStatusCodes. Validation error: The value is not in the list of acceptable values 

